In [233]:
from importlib import reload
from semantic_search import store
reload(store)
from semantic_search.store import LocalEmbeddingModel

model = LocalEmbeddingModel(
    model_name='sentence-transformers/all-MiniLM-L6-v2',  # 'sentence-transformers/all-MiniLM-L6-v2',
    chunk_size=512,
    chunk_overlap=64,
    batch_size=8
)

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/717k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Using device: cpu


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [234]:
import torch

tmp_input = "Using pooling, it generates from a variable sized sentence a fixed sized sentence embedding. This layer also allows to use the CLS token if it is returned by the underlying word embedding model. You can concatenate multiple poolings together."

inputs = [tmp_input*10, tmp_input*3, tmp_input, tmp_input*30]

_, all_chunks_encoded = model.chunk_and_encode(inputs, progress_bar=True)

# Flatten encoded
tmp = {}
for single_text_encoded in all_chunks_encoded:
    for k, v in single_text_encoded.items():
        if k not in tmp:
            tmp[k] = []
        tmp[k].append(v)
encoded_flattened = {k: torch.cat(v) for k, v in tmp.items()}

# Get embeddings for all chunks
print(f"Generating embeddings for {len(list(encoded_flattened.values())[0])} chunks...")
embeddings = model.get_embeddings(encoded_flattened, progress_bar=True)

Chunking and encoding: 100%|██████████| 4/4 [00:00<00:00, 585.61it/s]


Generating embeddings for 8 chunks...


Generating embeddings: 100%|██████████| 1/1 [00:02<00:00,  2.76s/it]


In [231]:
from torch.nn import functional as F

_, encoded = model.chunk_and_encode(tmp_input*100)
emb = model.get_embeddings(encoded)


encoded_input = model.tokenizer(tmp_input*100, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model.model(**encoded_input)

# Perform pooling
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)



In [232]:
emb[0] == sentence_embeddings[0]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

## Dev for new cleaner store setup

In [24]:
from importlib import reload
from semantic_search import store
reload(store)
from semantic_search.store import LocalEmbeddingModel, FAISSDocumentStore
from tqdm import tqdm

In [2]:
ds = FAISSDocumentStore(db_dir='/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/db/hybrid-dev')
assert ds.load_store()

2025-05-12 22:20:02.927660: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-12 22:20:04.794705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using device: cpu
Loaded index with 41220 vectors


In [11]:
ds.chunk_store[list(set(['doc_id']))]

,doc_id
chunk_id,
0,W2194775991
1,W2194775991
2,W1686810756
3,W4385245566
4,W4385245566
...,...
41215,W4317940424
41216,W4386977288
41217,W4387995378


In [3]:
ds.search('What is the main idea of the paper?', top_k=10)

[]

## Dev for new hybrid search

In [1]:
from semantic_search.store.store import FAISSDocumentStore

ds = FAISSDocumentStore(db_dir='/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/db/sentence-transformers_all-MiniLM-L6-v2')
assert ds.load_store()

/cluster/home/lcarretero/python_envs/dsl-research-assistant/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


KeyError: 'store_raw_documents'

In [20]:
ds.search('What is the main idea of the paper?', top_k=10)

[{'rank': 1,
  'score': 0.55700177,
  'doc_id': 'W3173515979',
  'doi': '10.1109/cvprw53098.2021.00286',
  'topic': 'Advanced Image Processing Techniques'},
 {'rank': 2,
  'score': 0.5426627,
  'doc_id': 'W3024079478',
  'doi': '10.1109/wacv45572.2020.9093474',
  'topic': 'Face recognition and analysis'},
 {'rank': 3,
  'score': 0.53668606,
  'doc_id': 'W3178812510',
  'doi': '10.1109/icicsp55539.2022.10050624',
  'topic': 'Advanced Neural Network Applications'},
 {'rank': 4,
  'score': 0.4742528,
  'doc_id': 'W4386066489',
  'doi': '10.1109/cvpr52729.2023.01670',
  'topic': 'Robotics and Sensor-Based Localization'},
 {'rank': 5,
  'score': 0.46302596,
  'doc_id': 'W3085567564',
  'doi': '10.1109/jsen.2020.3023486',
  'topic': 'Non-Invasive Vital Sign Monitoring'},
 {'rank': 6,
  'score': 0.46051693,
  'doc_id': 'W2766042539',
  'doi': '10.1145/3123266.3123451',
  'topic': 'Anomaly Detection Techniques and Applications'},
 {'rank': 7,
  'score': 0.45910347,
  'doc_id': 'W2740982616',
 

In [22]:
ds.chunk_store.head()

,doc_id,doi,topic
chunk_id,,,
0,W2194775991,10.1109/cvpr.2016.90,Advanced Neural Network Applications
1,W2194775991,10.1109/cvpr.2016.90,Advanced Neural Network Applications
2,W1686810756,nan,Advanced Vision and Imaging
3,W4385245566,10.48550/arxiv.1706.03762,Natural Language Processing Techniques
4,W4385245566,10.48550/arxiv.1706.03762,Natural Language Processing Techniques
